In [ ]:
import cv2
import numpy as np

# Load YOLO
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Get output layer names
unconnected_out_layers = net.getUnconnectedOutLayers()
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in unconnected_out_layers.flatten()]

# Load COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load and preprocess the image
img = cv2.imread("mmm.png")
img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Prepare the image for YOLO
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

class_ids = []
boxes = []
confidences = []

for out in outs:
    for detection in out:
        detection = np.array(detection)

        if len(detection.shape) == 1: 
            detection = np.expand_dims(detection, axis=0)
        
        for obj in detection:
            if obj.ndim != 1:
                print(f"Skipping object due to unexpected ndim: {obj.ndim}")
                continue

            if obj.size < 5:
                print("Object size is smaller than expected")
                continue

            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

#To reduce duplicated boxes
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

# Draw bounding boxes and labels on the image
font = cv2.FONT_HERSHEY_PLAIN

for i in indices.flatten():
    x, y, w, h = boxes[i]
    label = str(classes[class_ids[i]])
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(img, label, (x, y + 30), font, 1, (255, 255, 255), 2)

# Display the image
cv2.imshow("image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


2024-09-08 16:10:20.090 Python[2911:135374] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
